In [1]:
# useful tutorial how use GO: https://github.com/mousepixels/sanbomics_scripts/blob/main/GO_in_python.ipynb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# extract from the NCBI database the human genes
# https://www.ncbi.nlm.nih.gov/gene/
# "9606"[Taxonomy ID] AND alive[property] AND genetype protein coding[Properties] 

ERROR: Could not find a version that satisfies the requirement python==3.9 (from versions: none)
ERROR: No matching distribution found for python==3.9


In [2]:
!pip install goatools

#/home/rickbook/mambaforge/envs/pytorch2/lib/python3.10/site-packages

  Using cached goatools-1.3.1-py3-none-any.whl (15.8 MB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Using cached XlsxWriter-3.1.0-py3-none-any.whl (152 kB)
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached pydot-1.4.2-py2.py3-none-any.whl (21 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=16fba2420a8b1ee31b621452a60a633659362fd2b22aedceeafc428852986789
  Stored in directory: /home/rickbook/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


Extract the gene from the txt file with goatools

In [5]:
!python /home/rickbook/mambaforge/envs/pytorch2/bin/ncbi_gene_results_to_python.py -o genes_ncbi_mus_musculus_proteincoding.py gene_result.txt

      20,719 lines READ:  gene_result.txt
      20,700 geneids WROTE: genes_ncbi_mus_musculus_proteincoding.py


In [2]:
from genes_ncbi_mus_musculus_proteincoding import GENEID2NT as GeneID2nt_mus

from goatools.base import download_go_basic_obo
from goatools.base import download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS


obo_fname = download_go_basic_obo()
fin_gene2go = download_ncbi_associations()
obodag = GODag("go-basic.obo")

  EXISTS: go-basic.obo
  EXISTS: gene2go
go-basic.obo: fmt(1.2) rel(2023-04-01) 46,575 Terms


In [3]:
mapper = {}

for key in GeneID2nt_mus:
    mapper[GeneID2nt_mus[key].Symbol] = GeneID2nt_mus[key].GeneID
    
inv_map = {v: k for k, v in mapper.items()}

In [4]:
# objanno = Gene2GoReader(fin_gene2go, taxids=[9606])
geneid2gos_human = Gene2GoReader(fin_gene2go, taxids=[9606])

HMS:0:00:05.386132 342,553 annotations, 20,749 genes, 18,982 GOs, 1 taxids READ: gene2go 


In [5]:
ns2assoc = geneid2gos_human.get_ns2assc()

In [6]:
goeaobj = GOEnrichmentStudyNS(
        GeneID2nt_mus.keys(), # List of mouse protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method


Load BP Ontology Enrichment Analysis ...
 82% 16,984 of 20,700 population items found in association

Load CC Ontology Enrichment Analysis ...
 87% 18,105 of 20,700 population items found in association

Load MF Ontology Enrichment Analysis ...
 85% 17,570 of 20,700 population items found in association


In [7]:
GO_items = []

temp = goeaobj.ns2objgoea['BP'].assoc
for item in temp:
    GO_items += temp[item]
    

temp = goeaobj.ns2objgoea['CC'].assoc
for item in temp:
    GO_items += temp[item]
    

temp = goeaobj.ns2objgoea['MF'].assoc
for item in temp:
    GO_items += temp[item]

In [8]:
from itertools import chain

def prt_assc_counts(ns2assc):
    """Print the number of genes and GO IDs in an association"""
    for nspc, gene2goids in sorted(ns2assc.items()):
        print("{NS} {N:6,} genes, {GOs:6,} GOs".format(
            NS=nspc, N=len(gene2goids), GOs=len(set.union(*gene2goids.values()))))
        
ns2assc_all = geneid2gos_human.get_ns2assc(True)
prt_assc_counts(ns2assc_all)

BP 18,649 genes, 12,533 GOs
CC 19,755 genes,  1,790 GOs
MF 18,259 genes,  4,611 GOs


In [9]:
list(ns2assc_all.items())

[('MF',
  {2: {'GO:0002020',
    'GO:0004866',
    'GO:0004867',
    'GO:0005102',
    'GO:0005515',
    'GO:0019838',
    'GO:0019899',
    'GO:0019959',
    'GO:0019966',
    'GO:0042802',
    'GO:0043120',
    'GO:0048306',
    'GO:0048403',
    'GO:0048406'},
   9: {'GO:0004060'},
   10: {'GO:0004060', 'GO:0005515'},
   12: {'GO:0003677', 'GO:0004867', 'GO:0005515'},
   13: {'GO:0003824',
    'GO:0004806',
    'GO:0005515',
    'GO:0016298',
    'GO:0017171',
    'GO:0019213'},
   14: {'GO:0005515', 'GO:0008201'},
   15: {'GO:0004059', 'GO:0004060', 'GO:0005515', 'GO:0071889'},
   16: {'GO:0000049',
    'GO:0002161',
    'GO:0002196',
    'GO:0004813',
    'GO:0005524',
    'GO:0008270',
    'GO:0016597',
    'GO:0045182'},
   18: {'GO:0003867',
    'GO:0030170',
    'GO:0032145',
    'GO:0034386',
    'GO:0042802',
    'GO:0046872',
    'GO:0047298',
    'GO:0051536'},
   19: {'GO:0005102',
    'GO:0005515',
    'GO:0005524',
    'GO:0005548',
    'GO:0008320',
    'GO:0015485',
 

Extract and save the go term for each gene as a tensor with the respective id

In [29]:
import torch

number_of_amino_acids = 12533
def get_dict_all_go_tensor(ns2assc_all):
    """This function returns a dictionary with all the go terms encoded
    in a tensor with a vocab of 12533 (number of amino acids).
    
    """
    dictionary_all = {}
    for fun in ns2assc_all.keys():
        dictionary = {}
        for i in ns2assc_all[fun].keys():
            temp = []
            try:
                for j in list(ns2assc_all['MF'][i]):
                    temp.append(hash(j) % number_of_amino_acids)
                dictionary[i] = torch.from_numpy(np.array(temp)).type(torch.int16)
            except:
                pass
        dictionary_all[fun] = dictionary
    return dictionary_all

dictionary_all = get_dict_all_go_tensor(ns2assc_all)

# import pickle

# with open('proteins/go_term/dictionary_all_go_tensor.pkl', 'wb') as f:
#     pickle.dump(dictionary_all, f)
# with open('proteins/go_term/dictionary_all_go.pkl', 'wb') as f:
#     pickle.dump(ns2assc_all, f)

print('gene with maximum number of go term:', 
      max([len(dictionary_all[i][j]) for i in dictionary_all.keys() for j in dictionary_all[i].keys()]))

gene with maximum number of go term: 37


In [39]:
max_lenght = max([len(dictionary_all[i][j]) for i in dictionary_all.keys() for j in dictionary_all[i].keys()])

# torch.pad(dictionary_all['BP']['GO:0000001'], (0, max_lenght - len(dictionary_all['BP']['GO:0000001'])))

[torch.nn.functional.pad(dictionary_all[i][j], (0, max_lenght - len(dictionary_all[i][j])))
    for i in dictionary_all.keys() 
        for j in dictionary_all[i].keys()]

[tensor([ 5044,  8581, 11150,  5649,  6380, 12169,  4731,  1455,   663, 11493,
          9304,  1571,  3219,  9282,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0], dtype=torch.int16),
 tensor([242,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=torch.int16),
 tensor([5044,  242,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0], dtype=torch.int16),
 tensor([ 329, 5044, 1455,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          

In [42]:
import torch

number_of_amino_acids = 12533
max_lenght = max([len(dictionary_all[i][j]) for i in dictionary_all.keys() for j in dictionary_all[i].keys()])

def get_dict_all_go_tensor_padded(ns2assc_all):
    """This function returns a dictionary with all the go terms encoded
    in a tensor with a vocab of 12533 (number of amino acids) padded.
    
    """
    dictionary_all = {}
    for fun in ns2assc_all.keys():
        dictionary = {}
        for i in ns2assc_all[fun].keys():
            temp = []
            try:
                for j in list(ns2assc_all['MF'][i]):
                    temp.append(hash(j) % number_of_amino_acids)
                dictionary[i] = torch.nn.functional.pad(
                                            torch.from_numpy(
                                                    np.array(sorted(temp))
                                                ).type(torch.int16), (0, max_lenght - len(temp)))
            except:
                pass
        dictionary_all[fun] = dictionary
    return dictionary_all

dictionary_all_padded = get_dict_all_go_tensor_padded(ns2assc_all)

import pickle

with open('proteins/go_term/dictionary_all_go_tensor_padded.pkl', 'wb') as f:
    pickle.dump(dictionary_all_padded, f)

In [43]:
dictionary_all_padded

{'MF': {2: tensor([  663,  1455,  1571,  3219,  4731,  5044,  5649,  6380,  8581,  9282,
           9304, 11150, 11493, 12169,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0], dtype=torch.int16),
  9: tensor([242,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=torch.int16),
  10: tensor([ 242, 5044,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0], dtype=torch.int16),
  12: tensor([ 329, 1455, 5044,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,  